In [1]:
import BimatrixGame as BG
import globals as gl
import torch
import numpy as np
from environmentModelBase import Model, MixedStrategy, Strategy, StrategyType
import environmentModelBase as em
from learningBase import ReinforceAlgorithm
from neuralNetworkSimple import NNBase


In [2]:
np.random.seed(0)
gl.initialize()

In [ ]:
number_rounds=200


equilibria = []
low_cost_players = [Strategy(StrategyType.static, NNorFunc=em.myopic, name="myopic"), Strategy(
    StrategyType.static, NNorFunc=em.const, name="const", firstPrice=132), Strategy(StrategyType.static, NNorFunc=em.guess, name="guess", firstPrice=132)]
high_cost_players = [Strategy(StrategyType.static, NNorFunc=em.myopic, name="myopic"), Strategy(
    StrategyType.static, NNorFunc=em.const, name="const", firstPrice=132), Strategy(StrategyType.static, NNorFunc=em.guess, name="guess", firstPrice=132)]
bimatrixGame = BG.BimatrixGame(low_cost_players, high_cost_players)
# bimatrixGame.reset_matrix()
bimatrixGame.fill_matrix()
for round in range(number_rounds):
    print("Round", round, " of ", number_rounds)
    update = False

    low_cost_probabilities, high_cost_probabilities, low_cost_payoff, high_cost_payoff = bimatrixGame.compute_equilibria()
    print("equi: ", low_cost_probabilities, high_cost_probabilities,low_cost_payoff, high_cost_payoff)
    acceptable, agentPayoffs, advPayoffs, low_cost_player = BG.training([gl.lowCost, gl.highCost], advMixedStrategy=MixedStrategy(
            strategiesList=high_cost_players, probablitiesArray=high_cost_probabilities),targetPayoff= low_cost_payoff)
    if acceptable:
        update = True
        low_cost_players.append(low_cost_player)
        bimatrixGame.add_low_cost_row(agentPayoffs, advPayoffs)
        equilibria.append(
            [low_cost_probabilities, high_cost_probabilities, low_cost_payoff, high_cost_payoff])
        print(f"low cost player {low_cost_player._name} added")

        low_cost_probabilities, high_cost_probabilities, low_cost_payoff, high_cost_payoff = bimatrixGame.compute_equilibria()
        print("equi: ", low_cost_probabilities, high_cost_probabilities,low_cost_payoff, high_cost_payoff)
    acceptable, agentPayoffs, advPayoffs, high_cost_player = BG.training(
        [gl.highCost, gl.lowCost], advMixedStrategy=MixedStrategy(probablitiesArray= low_cost_probabilities, strategiesList=low_cost_players), targetPayoff= high_cost_payoff)

    if acceptable:
        update = True
        high_cost_players.append(high_cost_player)
        bimatrixGame.add_high_cost_col(advPayoffs, agentPayoffs)
        equilibria.append(
            [low_cost_probabilities, high_cost_probabilities, low_cost_payoff, high_cost_payoff])
        print(f"high cost player {high_cost_player._name} added")

        low_cost_probabilities, high_cost_probabilities, low_cost_payoff, high_cost_payoff = bimatrixGame.compute_equilibria()

    if update:
        gl.numEpisodes = gl.numEpisodesReset
    else:
        gl.numEpisodes += 1000

C:\Users\sjaha\Documents\EquiLearn\PGM_base\environmentModelBase.py:344: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  adversaryDist = Categorical(torch.tensor(self._strategyProbs))


Round 0  of  200
equi:  [0, 0, 1.0] [1.0, 0, 0] 154997.57716342332 79665.90160926129
